In [ ]:
from sklearn.metrics import r2_score
from datetime import date, timedelta, datetime
from itertools import chain, permutations
from matplotlib import pyplot as plt
from numpy import loadtxt
from numpy import matrix
from random import randint
from scipy import stats
from scipy.stats import ks_2samp
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.utils.extmath import cartesian
from statsmodels.distributions.empirical_distribution import ECDF
from time import time,strftime, gmtime
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import Reddit
import bisect
import functools
import glob
import math
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import more_itertools as mit
import networkx as nx
import numpy as np
import operator
import os
import pandas as pd
import pickle
import random
import re
import shutil
import statsmodels as sm
import time
import torch
import torch.utils.data
import warnings
import xgboost as xgb
import seaborn as sns
pd.set_option('display.max_columns', 150)


In [ ]:
df = pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None)

In [ ]:

df = df.loc[df['l_veh'].notna()]
dg = df.groupby(['measurement_tstamp','Detector_ID','dir','movements']).count().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','pair']]
dg = dg.rename(columns={"pair":"approach_obs"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left')
df = df[df["movements"]==df["approach_obs"]]
df['h_veh'] = df['h_veh'].fillna(0)
df['count'] = df['h_veh'] + df['l_veh']
df = df.drop(['l_veh','h_veh'],axis=1)
dg = df.groupby(['measurement_tstamp','Detector_ID','dir']).sum().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','count']]
dg = dg.rename(columns={"count":"approach_count"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').drop(['approach_obs'],axis=1)
# df = df[df['speed_up'].notna()]
# df = df[df['speed_dn'].notna()]
df['date'] = pd.to_datetime(df['measurement_tstamp']).dt.date

In [ ]:
df = df[['pair','measurement_tstamp','count','approach_count','date']].merge(pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None), how='right', on=['pair','measurement_tstamp'])
df = df.drop(['Unnamed: 0','l_veh','h_veh','movements'],axis=1)
df[['dir','move']] = df.Direction.str.split("_",expand=True)
df = df.merge(pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\Allowable_Movements.csv", delimiter=',', dtype=None).rename(columns={'movements':'allow_movements'}), on=['Detector_ID','dir'], how='left')
df

In [ ]:
# inter_list = list([6353,6354,6351,6814])
# df = df[df['Detector_ID'].isin(inter_list)]
df['sum_index'] = 0
df.loc[(df['Detector_ID']==6351)&((df['Direction']=='WB_RT')|(df['Direction']=='NB_TH')),'sum_index']=1
df.loc[(df['Detector_ID']==6814)&((df['Direction']=='WB_LT')|(df['Direction']=='EB_RT')),'sum_index']=2
df.loc[(df['Detector_ID']==6354)&((df['Direction']=='WB_RT')|(df['Direction']=='NB_TH')|(df['Direction']=='EB_LT')),'sum_index']=3
df.loc[(df['Detector_ID']==6351)&((df['Direction']=='WB_LT')|(df['Direction']=='EB_RT')|(df['Direction']=='SB_TH')),'sum_index']=4
df.loc[(df['Detector_ID']==6353)&((df['Direction']=='WB_RT')|(df['Direction']=='NB_TH')|(df['Direction']=='EB_LT')),'sum_index']=5
df.loc[(df['Detector_ID']==6354)&((df['Direction']=='WB_LT')|(df['Direction']=='EB_RT')|(df['Direction']=='SB_TH')),'sum_index']=6
df['nan_exist'] = 0
df.loc[df['count'].isna(),'nan_exist']=1

In [ ]:
dg = df[['measurement_tstamp','count','sum_index','nan_exist']].groupby(['measurement_tstamp','sum_index']).sum().reset_index()
dg = dg[dg['sum_index']>0]
dg = dg[dg['nan_exist']==0]
dg = dg.rename(columns={'count':'approach_count_dg','sum_index':'det_index'})
dg

In [ ]:
df['sum_index']=0
df.loc[(df['Detector_ID']==6814)&(df['dir']=='NB'),'det_index']=1
df.loc[(df['Detector_ID']==6351)&(df['dir']=='SB'),'det_index']=2
df.loc[(df['Detector_ID']==6351)&(df['dir']=='NB'),'det_index']=3
df.loc[(df['Detector_ID']==6354)&(df['dir']=='SB'),'det_index']=4
df.loc[(df['Detector_ID']==6354)&(df['dir']=='NB'),'det_index']=5
df.loc[(df['Detector_ID']==6353)&(df['dir']=='SB'),'det_index']=6
print(df['approach_count'].isna().sum(axis=0))
df = df.merge(dg[['measurement_tstamp','det_index','approach_count_dg']], on=['measurement_tstamp','det_index'], how='left')
df.approach_count.fillna(df.approach_count_dg, inplace=True)
df.drop(['approach_count_dg'], inplace=True, axis=1)
print(df['approach_count'].isna().sum(axis=0))

In [ ]:
print(df['avg_speed_up'].isna().sum(axis=0))
print(df['avg_speed_dn'].isna().sum(axis=0))
print(df['ref_speed_up'].isna().sum(axis=0))
print(df['ref_speed_dn'].isna().sum(axis=0))

In [ ]:
dq = df[['Up','year','month','weekday','hour','avg_speed_up','ref_speed_up','avg_speed_dn',
         'ref_speed_dn']].groupby(['Up','year','month','weekday','hour']).mean()
dq = dq.rename(columns={'avg_speed_up':"avg_speed_up_dq",'ref_speed_up':'ref_speed_up_dq','avg_speed_dn':'avg_speed_dn_dq',
                        'ref_speed_dn':'ref_speed_dn_dq'})
df = df.merge(dq,on=['Up','year','month','weekday','hour'])
df.avg_speed_up.fillna(df.avg_speed_up_dq, inplace=True)
df.avg_speed_dn.fillna(df.avg_speed_dn_dq, inplace=True)
df.ref_speed_up.fillna(df.ref_speed_up_dq, inplace=True)
df.ref_speed_dn.fillna(df.ref_speed_dn_dq, inplace=True)


print(df['avg_speed_up'].isna().sum(axis=0))
print(df['avg_speed_dn'].isna().sum(axis=0))
print(df['ref_speed_up'].isna().sum(axis=0))
print(df['ref_speed_dn'].isna().sum(axis=0))

In [ ]:
inter_list = list([6353,6354,6351,6814])
dg = df[df['Detector_ID'].isin(inter_list)]
print(dg['speed_up'].isna().sum(axis=0))
print(dg['speed_dn'].isna().sum(axis=0))
print(dg['avg_speed_up'].isna().sum(axis=0))
print(dg['avg_speed_dn'].isna().sum(axis=0))
print(dg['ref_speed_up'].isna().sum(axis=0))
print(dg['ref_speed_dn'].isna().sum(axis=0))

In [ ]:
df.to_csv('C:\\Users\\szahedi1\\Amir_Thesis\\Data_all.csv', sep=',')

In [1]:
from sklearn.metrics import r2_score
from datetime import date, timedelta, datetime
from itertools import chain, permutations
from matplotlib import pyplot as plt
from numpy import loadtxt
from numpy import matrix
from random import randint
from scipy import stats
from scipy.stats import ks_2samp
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.utils.extmath import cartesian
from statsmodels.distributions.empirical_distribution import ECDF
from time import time,strftime, gmtime
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import Reddit
import bisect
import functools
import glob
import math
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import more_itertools as mit
import networkx as nx
import numpy as np
import operator
import os
import pandas as pd
import pickle
import random
import re
import shutil
import statsmodels as sm
import time
import torch
import torch.utils.data
import warnings
import xgboost as xgb
import seaborn as sns
pd.set_option('display.max_columns', 150)

In [2]:
df = pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\Data_all.csv", delimiter=',' ,dtype=None)
dq = df[df.avg_speed_up.notna()].drop(['pair','count','date', 'Up','Dn','Direction','Exclusive_Lane',
                                       'roadname_up','xd_up','tmc_up', 'roadname_dn','xd_dn','tmc_dn',
                                       'lanes_dn','speed_dn','avg_speed_dn','ref_speed_dn','Intersection Name',
                                       'sum_index','nan_exist','det_index','move','avg_speed_up_dq',
                                       'ref_speed_up_dq','avg_speed_dn_dq','ref_speed_dn_dq','measurement_tstamp',
                                      'approach_count','speed_up','ref_speed_up','Unnamed: 0'],axis=1)
dum_dq = pd.get_dummies(dq, columns=['year','month','weekday','day',"hour",'quarter','Detector_ID','dir','lanes_up' ,
                                     'allow_movements'])

C:\Users\szahedi1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,19,23,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dum_dq = dum_dq+0.01

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import keras
pd.set_option('display.max_columns', 150)

# from tensorflow.keras.layers import Activation
# from keras.layers.normalization import BatchNormalization


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Activation, Dropout, Dense, BatchNormalization, LeakyReLU

from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers

#from keras.layers import LeakyReLU


from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
from sklearn.metrics import r2_score
from random import randint
import time

In [5]:
result = pd.DataFrame(columns = ["Train_MSE","Validation_MSE","Train_MAE","Validation_MAE",
                                 "Train_MAPE","Validation_MAPE","Train_R_Squared","Validation_R_Squared"])
Epochs=2

train, test = train_test_split(dum_dq, test_size=0.1, random_state=42, shuffle=True)


X_train=train.drop(['avg_speed_up'],axis=1)
X_test=test.drop(['avg_speed_up'],axis=1)

y_train=train['avg_speed_up']
y_test=test['avg_speed_up']

scaler = preprocessing.StandardScaler().fit(X_train)
X_train= scaler.transform(X_train)
X_test = scaler.transform(X_test)
model = Sequential()

model.add(Dense(64, input_dim=X_train.shape[1], use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.10))

model.add(Dense(64, use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.10))

model.add(Dense(64, use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.10))

model.add(Dense(64, use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.10))

model.add(Dense(1, activation='linear'))

model.summary()
model.compile(loss='mae', optimizer='adam', metrics=['mse','mae','mape'])
history = model.fit(X_train, y_train, epochs=Epochs, batch_size=2048,  verbose=1, validation_data = (X_test, y_test))


result = result.append({"Train_MSE": history.history['mse'][Epochs-1],
                        "Validation_MSE": history.history['val_mse'][Epochs-1],
                        "Train_MAE": history.history['mae'][Epochs-1],
                        "Validation_MAE": history.history['val_mae'][Epochs-1],
                        "Train_MAPE": history.history['mape'][Epochs-1],
                        "Validation_MAPE": history.history['val_mape'][Epochs-1],
                        "Training_Set_Size": X_train.shape[0]},
                       ignore_index=True)
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                6848      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4096      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4

InternalError:  Blas GEMM launch failed : a.shape=(2048, 107), b.shape=(107, 64), m=2048, n=64, k=107
	 [[node sequential/dense/MatMul (defined at <ipython-input-5-5d96baf5106f>:43) ]] [Op:__inference_train_function_2057]

Function call stack:
train_function


In [ ]:
dum_dq.dtypes

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
df[(df['Up']==1595153899)&(df['year']==2020)&(df['weekday']==0)&(df['hour']==5)].avg_speed_up.unique()

In [ ]:
df[(df['Up']==1595153899)&(df['year']==2020)&(df['weekday']==0)&(df['hour']==5)&(df['avg_speed_up']==11)].month.unique()

In [ ]:
dg = df[~df['approach_count'].isna()].drop(['pair','count','date', 'Up','Dn','Direction','Exclusive_Lane',
                                            'roadname_up','xd_up','tmc_up', 'roadname_dn','xd_dn','tmc_dn',
                                            'lanes_dn','speed_dn','avg_speed_dn','ref_speed_dn','Intersection Name',
                                            'sum_index','nan_exist','det_index','move'],axis=1)


In [ ]:
dg = dg.drop_duplicates()
dg['measurement_tstamp'] = pd.to_datetime(dg.measurement_tstamp)
dg['week'] = dg.measurement_tstamp.dt.week
dg

In [ ]:
dum_dg = pd.get_dummies(dg, columns=["hour",'quarter','year','month','weekday','day','Detector_ID','dir','lanes_up' ,'allow_movements'])
dq = dg[['Detector_ID','year']]
dum_dg = dum_dg.join(dq).drop(['measurement_tstamp','week','year'],axis=1)
dum_dg

In [ ]:
i=0
for loc in dum_dg.Detector_ID.unique().tolist():
    i=i+1
    train=dum_dg.loc[dum_dg['Detector_ID']!=loc]
    testt=dum_dg.loc[dum_dg['Detector_ID']==loc]
    trainn, test = train_test_split(testt, test_size=0.1, random_state=42, shuffle=True)
    train=train.append(trainn)
    
    
    X_train=train.drop(['approach_count','Detector_ID'],axis=1)
    X_test=test.drop(['approach_count','Detector_ID'],axis=1)
    
    y_train=train['approach_count']
    y_test=test['approach_count']
    
    regressor = xgb.XGBRegressor(n_estimators=100,reg_lambda=1, gamma=0,max_depth=3)
    regressor.fit(X_train, y_train)
    res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)
    if (i==1):
        results=res
    else:
        results=pd.concat([results,res])
if (len(results.loc[results['actual']-results['approach_count']!=0])==0):
    print('Done')
else:
    print('there is a problem')

In [ ]:
# count
result = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAPE"])
for loc in dum_dg.Detector_ID.unique().tolist():
    result=result.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'approach_count'], results.loc[results['Detector_ID']==loc,'pred'] ),
                        "XGBoost_MAPE": (abs(results.loc[results['Detector_ID']==loc,'approach_count']-results.loc[results['Detector_ID']==loc,'pred'])/results.loc[results['Detector_ID']==loc,'approach_count']).mean()*100
                        },
                       ignore_index=True)
    
results['pro_act'] = results['approach_count']/results['approach_count']
results['pro_est'] = results['pred']/results['approach_count']

# proportion
result_p = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAE"])
for loc in dum_dg.Detector_ID.unique().tolist():
    result_p=result_p.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'pro_act'], results.loc[results['Detector_ID']==loc,'pro_est'] ),
                        "XGBoost_MAE": (abs(results.loc[results['Detector_ID']==loc,'pro_act']-results.loc[results['Detector_ID']==loc,'pro_est'])).mean()*100
                        },
                       ignore_index=True)

In [ ]:
result

# Approach Volume Estimation

## XGBoost

In [ ]:
df = pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None)

df = df.loc[df['l_veh'].notna()]
dg = df.groupby(['measurement_tstamp','Detector_ID','dir','movements']).count().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','pair']]
dg = dg.rename(columns={"pair":"approach_obs"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left')
df = df[df["movements"]==df["approach_obs"]]
df['h_veh'] = df['h_veh'].fillna(0)
df['count'] = df['h_veh'] + df['l_veh']
df = df.drop(['l_veh','h_veh'],axis=1)
dg = df.groupby(['measurement_tstamp','Detector_ID','dir']).sum().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','count']]
dg = dg.rename(columns={"count":"approach_count"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').drop(['approach_obs'],axis=1)
# df = df[df['speed_up'].notna()]
# df = df[df['speed_dn'].notna()]
df['date'] = pd.to_datetime(df['measurement_tstamp']).dt.date

df = df[['pair','measurement_tstamp','count','approach_count','date']].merge(pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None), how='right', on=['pair','measurement_tstamp'])
df = df.drop(['Unnamed: 0','l_veh','h_veh','movements'],axis=1)
df[['dir','move']] = df.Direction.str.split("_",expand=True)
df = df.merge(pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\Allowable_Movements.csv", delimiter=',', dtype=None).rename(columns={'movements':'allow_movements'}), on=['Detector_ID','dir'], how='left')


In [ ]:
dg = df[~df['approach_count'].isna()].drop(['pair','count','date', 'Up','Dn','Direction','Exclusive_Lane',
                                            'roadname_up','xd_up','tmc_up', 'roadname_dn','xd_dn','tmc_dn',
                                            'lanes_dn','speed_dn','avg_speed_dn','ref_speed_dn','Intersection Name',
                                            'move'],axis=1)
dg = dg[~dg['speed_up'].isna()]
dg = dg.drop_duplicates()
dg['measurement_tstamp'] = pd.to_datetime(dg.measurement_tstamp)
dg['week'] = dg.measurement_tstamp.dt.week

dum_dg = pd.get_dummies(dg, columns=["hour",'quarter','year','month','weekday','day','Detector_ID','dir','lanes_up' ,
                                     'allow_movements'])
dq = dg[['Detector_ID','year']]
dum_dg = dum_dg.join(dq).drop(['measurement_tstamp','week','year'],axis=1)

In [ ]:
i=0
for loc in dum_dg.Detector_ID.unique().tolist():
    i=i+1
    train=dum_dg.loc[dum_dg['Detector_ID']!=loc]
    testt=dum_dg.loc[dum_dg['Detector_ID']==loc]
    trainn, test = train_test_split(testt, test_size=0.1, random_state=42, shuffle=True)
    train=train.append(trainn)
    
    
    X_train=train.drop(['approach_count','Detector_ID'],axis=1)
    X_test=test.drop(['approach_count','Detector_ID'],axis=1)
    
    y_train=train['approach_count']
    y_test=test['approach_count']
    
    regressor = xgb.XGBRegressor(n_estimators=100,reg_lambda=1, gamma=0,max_depth=3)
    regressor.fit(X_train, y_train)
    res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)
    if (i==1):
        results=res
    else:
        results=pd.concat([results,res])
if (len(results.loc[results['actual']-results['approach_count']!=0])==0):
    print('Done')
else:
    print('there is a problem')
    
# count
result = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAPE"])
for loc in dum_dg.Detector_ID.unique().tolist():
    result=result.append({"Detector_ID":str(loc),
                          "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'approach_count'],
                                                       "XGBoost_MAPE": (abs(results.loc[results['Detector_ID']==loc,
                                                                                        'approach_count']-results.loc[results['Detector_ID']==loc,
                                                                                                                      'pred'])/results.loc[results['Detector_ID']==loc,
                                                                                                                                           'approach_count']).mean()*100},
                                                       ignore_index=True)
    
results['pro_act'] = results['approach_count']/results['approach_count']
results['pro_est'] = results['pred']/results['approach_count']

# proportion
result_p = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAE"])
for loc in dum_dg.Detector_ID.unique().tolist():
    result_p=result_p.append({"Detector_ID":str(loc),
                              "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'pro_act'],
                                                           "XGBoost_MAE": (abs(results.loc[results['Detector_ID']==loc,
                                                                                           'pro_act']-results.loc[results['Detector_ID']==loc,'pro_est'])).mean()*100
                        },
                       ignore_index=True)

In [ ]:
result

## NN Model

In [ ]:
df = pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None)

df = df.loc[df['l_veh'].notna()]
dg = df.groupby(['measurement_tstamp','Detector_ID','dir','movements']).count().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','pair']]
dg = dg.rename(columns={"pair":"approach_obs"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left')
df = df[df["movements"]==df["approach_obs"]]
df['h_veh'] = df['h_veh'].fillna(0)
df['count'] = df['h_veh'] + df['l_veh']
df = df.drop(['l_veh','h_veh'],axis=1)
dg = df.groupby(['measurement_tstamp','Detector_ID','dir']).sum().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','count']]
dg = dg.rename(columns={"count":"approach_count"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').drop(['approach_obs'],axis=1)
# df = df[df['speed_up'].notna()]
# df = df[df['speed_dn'].notna()]
df['date'] = pd.to_datetime(df['measurement_tstamp']).dt.date

df = df[['pair','measurement_tstamp','count','approach_count','date']].merge(pd.read_csv("D:/Amir_Nohekhan/data.csv", delimiter=',' ,dtype=None), how='right', on=['pair','measurement_tstamp'])
df = df.drop(['Unnamed: 0','l_veh','h_veh','movements'],axis=1)
df[['dir','move']] = df.Direction.str.split("_",expand=True)
df = df.merge(pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\Allowable_Movements.csv", delimiter=',', dtype=None).rename(columns={'movements':'allow_movements'}), on=['Detector_ID','dir'], how='left')


In [ ]:
dg = df[~df['approach_count'].isna()].drop(['pair','count','date', 'Up','Dn','Direction','Exclusive_Lane',
                                            'roadname_up','xd_up','tmc_up', 'roadname_dn','xd_dn','tmc_dn',
                                            'lanes_dn','speed_dn','avg_speed_dn','ref_speed_dn','Intersection Name',
                                            'move'],axis=1)
dg = dg[~dg['speed_up'].isna()]
dg = dg[~dg['avg_speed_up'].isna()]

dg = dg.drop_duplicates()
dg['measurement_tstamp'] = pd.to_datetime(dg.measurement_tstamp)
dg['week'] = dg.measurement_tstamp.dt.week

dum_dg = pd.get_dummies(dg, columns=["hour",'quarter','year','month','weekday','day','Detector_ID','dir','lanes_up' ,
                                     'allow_movements'])
dq = dg[['Detector_ID','year']]
dum_dg = dum_dg.join(dq).drop(['measurement_tstamp','week','year'],axis=1)

In [ ]:
dg[(dg['Detector_ID']==7047)&(dg['dir']=="NB")]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import keras
pd.set_option('display.max_columns', 150)

# from tensorflow.keras.layers import Activation
# from keras.layers.normalization import BatchNormalization


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Activation, Dropout, Dense, BatchNormalization, LeakyReLU

from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers

#from keras.layers import LeakyReLU


from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn import preprocessing
from sklearn.metrics import r2_score
from random import randint
import time

In [ ]:
result = pd.DataFrame(columns = ["Intersection_ID","Train_MSE","Validation_MSE","Train_MAE","Validation_MAE",
                                 "Train_MAPE","Validation_MAPE","Train_R_Squared","Validation_R_Squared"])
Epochs=20
i=0
for loc in dum_dg.Detector_ID.unique().tolist():
    i=i+1
    train=dum_dg.loc[dum_dg['Detector_ID']!=loc]
    testt=dum_dg.loc[dum_dg['Detector_ID']==loc]
    trainn, test = train_test_split(testt, test_size=0.1, random_state=42, shuffle=True)
    train=train.append(trainn)
    
    
    X_train=train.drop(['approach_count','Detector_ID'],axis=1)
    X_test=test.drop(['approach_count','Detector_ID'],axis=1)
    
    y_train=train['approach_count']
    y_test=test['approach_count']
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train= scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    model = Sequential()
    
    model.add(Dense(128, input_dim=110, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    #model.add(Dropout(0.10))
    
    model.add(Dense(128, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    #model.add(Dropout(0.10))
    
    model.add(Dense(128, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    #model.add(Dropout(0.10))
    
    model.add(Dense(128, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    #model.add(Dropout(0.10))
    
    model.add(Dense(1, activation='linear'))
    
    model.summary()
    model.compile(loss='mae', optimizer='adam', metrics=['mse','mae','mape'])
    history = model.fit(X_train, y_train, epochs=Epochs, batch_size=128,  verbose=1, validation_data = (X_test, y_test))
    
    
    result = result.append({"Intersection_ID": loc,
                            "Train_MSE": history.history['mse'][Epochs-1],
                            "Validation_MSE": history.history['val_mse'][Epochs-1],
                            "Train_MAE": history.history['mae'][Epochs-1],
                            "Validation_MAE": history.history['val_mae'][Epochs-1],
                            "Train_MAPE": history.history['mape'][Epochs-1],
                            "Validation_MAPE": history.history['val_mape'][Epochs-1],
                            "Training_Set_Size": X_train.shape[0]},
                           ignore_index=True)
    print(history.history.keys())
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
result

In [ ]:
#df[df['Detector_ID']==6814].groupby(['Detector_ID','dir']).count()

In [ ]:
#df[df['Detector_ID']==6351].groupby(['Detector_ID','dir']).count()

In [ ]:
df['dn_dir'] = ['NB' if x =="NB_TH" else 'NB' if x =="WB_RT" else 'NB' if x =="EB_LT" else
                'SB' if x =="SB_TH" else 'SB' if x =="EB_RT" else 'SB' if x =="WB_LT" else
                'EB' if x =="EB_TH" else 'EB' if x =="NB_RT" else 'EB' if x =="SB_LT" else 'WB'for x in df["Direction"]]
dg = df.groupby(['Detector_ID','measurement_tstamp','dn_dir']).count().reset_index()
dg = dg[dg['pair']==3][['Detector_ID','measurement_tstamp','dn_dir']]
df = df.merge(dg, how='inner', on=['Detector_ID','measurement_tstamp','dn_dir'])
dg = df.groupby(['Detector_ID','measurement_tstamp','dn_dir']).sum().reset_index()
dg = dg[['Detector_ID','measurement_tstamp','dn_dir','count']].rename(columns={'count':'dn_count'})
df = df.merge(dg, how='inner', on=['Detector_ID','measurement_tstamp','dn_dir'])
df = df.dropna()
df['measurement_tstamp'] = pd.to_datetime(df.measurement_tstamp)
df['week'] = df.measurement_tstamp.dt.week

dg = df.groupby(['Detector_ID','year','week']).count().reset_index()
dh = dg.groupby(['Detector_ID']).idxmax().reset_index()
for i in range(dh.shape[0]):
    dh.iloc[i,1] = dg.iloc[dh.iloc[i,3],1]
    dh.iloc[i,2] = dg.iloc[dh.iloc[i,3],2]
    
dh = dh[['Detector_ID','year','week']]
dh


In [ ]:
df.head()

In [ ]:
#df.to_csv('C:\\Users\\szahedi1\\Amir_Thesis\\Data.csv', sep=',')

In [ ]:
dum_df = pd.get_dummies(df, columns=["hour",'quarter','year','month','weekday','day','week',
                                     'Detector_ID','dir','move','lanes_up','lanes_dn'] )
dq = df[['Detector_ID','year','week']]
dum_df = dum_df.join(dq).drop(['roadname_up','roadname_dn','pair','measurement_tstamp','Up','Dn','Direction','xd_up','tmc_up','xd_dn','tmc_dn',
                               'Intersection Name','date','dn_dir','movements'],axis=1)
dum_df.head()

In [ ]:
i=0
for loc in dum_df.Detector_ID.unique().tolist():
    i=i+1
    train=dum_df.loc[dum_df['Detector_ID']!=loc]
    train=train.append(dum_df.loc[(dum_df['Detector_ID']==loc)&
                              (dum_df['year']==dh.loc[dh['Detector_ID']==loc,'year'].item())&
                              (dum_df['week']==dh.loc[dh['Detector_ID']==loc,'week'].item())])
    test=dum_df.loc[dum_df['Detector_ID']==loc]
    
    X_train=train.drop(['count','Detector_ID','year','week'],axis=1)
    X_test=test.drop(['count','Detector_ID','year','week'],axis=1)
    
    y_train=train['count']
    y_test=test['count']
    
    regressor = xgb.XGBRegressor(n_estimators=100,reg_lambda=1, gamma=0,max_depth=3)
    regressor.fit(X_train, y_train)
    res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)
    if (i==1):
        results=res
    else:
        results=pd.concat([results,res])
if (len(results.loc[results['actual']-results['count']!=0])==0):
    print('Done')
else:
    print('there is a problem')

In [ ]:
# count
result = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAPE"])
for loc in dum_df.Detector_ID.unique().tolist():
    result=result.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'count'], results.loc[results['Detector_ID']==loc,'pred'] ),
                        "XGBoost_MAPE": (abs(results.loc[results['Detector_ID']==loc,'count']-results.loc[results['Detector_ID']==loc,'pred'])/results.loc[results['Detector_ID']==loc,'count']).mean()*100
                        },
                       ignore_index=True)
    
results['pro_act'] = results['count']/results['approach_count']
results['pro_est'] = results['pred']/results['approach_count']

# proportion
result_p = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAE"])
for loc in dum_df.Detector_ID.unique().tolist():
    result_p=result_p.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'pro_act'], results.loc[results['Detector_ID']==loc,'pro_est'] ),
                        "XGBoost_MAE": (abs(results.loc[results['Detector_ID']==loc,'pro_act']-results.loc[results['Detector_ID']==loc,'pro_est'])).mean()*100
                        },
                       ignore_index=True)

In [ ]:
result

In [ ]:
dn = pd.DataFrame()
for loc in dum_df.Detector_ID.unique().tolist():
    i=i+1
    dn=dn.append(dum_df.loc[(dum_df['Detector_ID']==loc)&
                            (dum_df['year']==dh.loc[dh['Detector_ID']==loc,'year'].item())&
                            (dum_df['week']==dh.loc[dh['Detector_ID']==loc,'week'].item())])
dn

In [ ]:
results

In [ ]:
results['count_diff']    = results['pred']-results['count']
results['count_diff_h0'] = (results['approach_count']*results['pro_m'])-results['count']
results['abs_count_diff']    = abs(results['count_diff'])
results['abs_count_diff_h0'] = abs(results['count_diff_h0'])
results.abs_count_diff.mean()

In [ ]:
results.abs_count_diff_h0.mean()

In [ ]:
results['count'].mean()

In [ ]:
results[['Detector_ID','count_diff','abs_count_diff','abs_count_diff_h0','count']].groupby(['Detector_ID']).mean()

In [ ]:


dh = dh[['Detector_ID','measurement_tstamp']]
dh = dh.rename(columns={"measurement_tstamp":"date"})
dh

In [ ]:
dh = dh.merge(df, how='inner', on=['Detector_ID','date'])
dh['pro'] = dh['count']/dh['approach_count']
dh.head()

In [ ]:
dh[(dh['Detector_ID']==7047) & (dh['Direction']=='WB_RT')]

In [ ]:
dh = dh.rename(columns={'count':'count_m', 'approach_count':'approach_count_m', 'dn_count':'dn_count_m', 'pro':'pro_m'})
df = df.merge(dh[['Detector_ID','pair','hour','quarter','count_m', 'approach_count_m', 'dn_count_m', 'pro_m']], how='left'
              , on=['Detector_ID','pair','hour','quarter'])
df

In [ ]:
df.dropna()

In [ ]:
df['count_est'] = df['approach_count']*df['pro']
df['mape'] = abs(df['count_est']-df['count'])/df['count']*100
df = df.dropna()
df

In [ ]:
df[['mape','move']].groupby(['move']).mean()

In [ ]:
df[(df['pair']=='(1595162956,1595392997)') & (df['hour']==8) & (df['quarter']==1)]

In [ ]:
det_list = [6353,6354,6351,6814]
dg = df[df.Detector_ID.isin(det_list)]
dg = dg.groupby(['Detector_ID','measurement_tstamp']).count().reset_index()
dg['amax'] = [6 if x == 6814 else 6 if x == 6351 else 12 for x in dg["Detector_ID"]]
dg

In [ ]:
det_list = [6353,6354,6351,6814]
dg = df[df.Detector_ID.isin(det_list)]
dg = dg.groupby(['measurement_tstamp']).count().reset_index()
dg['pair'].describe()

In [ ]:
det_list = [6353,6354,6351]
dg = df[df.Detector_ID.isin(det_list)]
dg = dg.groupby(['measurement_tstamp']).count().reset_index()
dg['pair'].describe()

In [ ]:
det_list = [6353,6354]
dg = df[df.Detector_ID.isin(det_list)]
dg = dg.groupby(['measurement_tstamp']).count().reset_index()
dg['pair'].describe()

In [ ]:
df[(df['Detector_ID']==6354) & (df['Direction']=='NB_TH') & (df['year']==2018) & (df['month']==5) & (df['day']==4)].to_csv("C:\\Users\\szahedi1\\Desktop\\6354.csv", sep=',',index=False)

In [ ]:
df[(df['Detector_ID']==6354) & (df['Direction']=='NB_TH') & (df['year']==2019) & (df['month']==8)].day.unique()

In [ ]:
date='2018-05-04'
df.loc[(df['Detector_ID']==6354)&(df['date'].astype(str)==date) & (df['Direction']=='NB_RT')]

In [ ]:

list_date

In [ ]:
Detector_ID = 7047
Direction = 'NB_LT'
list_date = df.loc[(df['Detector_ID']==Detector_ID) & (df['Direction']==Direction)].date.unique()
hue_order=list_date[1:50]
#palette = sns.color_palette("Spectral", len(hue_order))
fig = plt.figure(figsize=(40,10))
ax1 = fig.add_subplot(121)
sns.lineplot(x='time', y='count', hue='date', hue_order=hue_order, data= df.loc[(df['Detector_ID']==Detector_ID) & (df['date'].isin(hue_order)) & (df['Direction']==Direction)]
             , legend='full' , ci=None, ax=ax1) #,palette=palette
ax1.set_xlim(0,24)
ax1.set_title('Detector '+str(Detector_ID), fontweight='bold')
ax1.set_ylabel('Flow (veh/15-min)')
ax1.set_xlabel('Time of Day')

In [ ]:
Detector_ID = 7047
Direction = 'NB_TH'
list_date = df.loc[(df['Detector_ID']==Detector_ID) & (df['Direction']==Direction)].date.unique()
hue_order=list_date[20:40]
#palette = sns.color_palette("Spectral", len(hue_order))
fig = plt.figure(figsize=(40,10))
ax1 = fig.add_subplot(121)
sns.lineplot(x='time', y='count', hue='date', hue_order=hue_order, data= df.loc[(df['Detector_ID']==Detector_ID) & (df['date'].isin(hue_order)) & (df['Direction']==Direction)]
             , estimator='mean',legend='full' , ci=None, ax=ax1) #,palette=palette
ax1.set_xlim(0,24)
ax1.set_title('Detector '+str(Detector_ID), fontweight='bold')
ax1.set_ylabel('Flow (veh/15-min)')
ax1.set_xlabel('Time of Day')

In [ ]:
Detector_ID = 7047
Direction = 'NB_RT'
list_date = df.loc[(df['Detector_ID']==Detector_ID) & (df['Direction']==Direction)].date.unique()
hue_order=list_date[20:23]
#palette = sns.color_palette("Spectral", len(hue_order))
fig = plt.figure(figsize=(40,10))
ax1 = fig.add_subplot(121)
sns.lineplot(x='time', y='count', hue='date', hue_order=hue_order, data= df.loc[(df['Detector_ID']==Detector_ID) & (df['date'].isin(hue_order)) & (df['Direction']==Direction)]
             , estimator='mean',legend='full' , ci=None, ax=ax1) #,palette=palette
ax1.set_xlim(0,24)
ax1.set_title('Detector '+str(Detector_ID), fontweight='bold')
ax1.set_ylabel('Flow (veh/15-min)')
ax1.set_xlabel('Time of Day')

In [ ]:
#date='2019-11-21'
Detector_ID = 7047
Direction = 'NB_LT'

list_date = df.loc[(df['Detector_ID']==Detector_ID) & (df['Direction']==Direction)].date.unique()
df['time'] = df['hour']+df['quarter']/4
fig, ax1 = plt.subplots(figsize=(15,10))
i=0
for date in list_date:
    if i>3:
        break
    else:
        df.loc[(df['Detector_ID']==Detector_ID)&(df['date'].astype(str)==date) & (df['Direction']==Direction)].plot(ax=ax1,x='time', y='count',color='mediumseagreen',legend=None)
        i+=1
#dg.loc[dg['Station']==i].plot(ax=ax,x='hour', y='q75')
ax1.set_xlabel("Time of Day",size=15)
ax1.set_ylabel('Flow (veh/15-min)', color='black',size=15)
#     ax1.set_xticks(np.arange(0, 97, 4))
#     ax1.set_xticklabels(tt, rotation=90, fontsize=25)
ax1.set_xlim(0,24)
#plt.gcf().autofmt_xdate()
#ax1.xaxis.set_major_formatter(myFmt)
#plt.legend([bpr["boxes"][0],bpl["boxes"][0]], ['ANN', 'FSTGCN'], edgecolor='black', loc=9, prop={'size': 15})
#plt.legend(prop={'size': 15})
#ax1.legend(['Through','Left-Turn','Right-Turn'], prop={'size': 15})
plt.title('Detector '+str(Detector_ID),size=20)

In [ ]:
myFmt = mdates.DateFormatter('%H:%M:%S')

for i in tmc.tmc.unique().tolist():
    date='2019-02-20'
    fig, ax1 = plt.subplots(figsize=(15,10))
    #dg.loc[dg['Station']==i].plot(ax=ax,x='hour', y='q25')
    tmc.loc[(tmc['tmc']==i)&(tmc['date']==date)].plot(ax=ax1,x='time', y='FSTGCN',color='mediumseagreen',legend=None)
    tmc.loc[(tmc['tmc']==i)&(tmc['date']==date)].plot(ax=ax1,x='time', y='ANN', color='salmon',legend=None)
    tmc.loc[(tmc['tmc']==i)&(tmc['date']==date)].plot(ax=ax1,x='time', y='XGBoost', color='cornflowerblue',legend=None)
    tmc.loc[(tmc['tmc']==i)&(tmc['date']==date)].plot(ax=ax1,x='time', y='ATR_Counts', color='k',legend=None)
    #dg.loc[dg['Station']==i].plot(ax=ax,x='hour', y='q75')
    ax1.set_xlabel("Time of Day",size=15)
    ax1.set_ylabel('Flow (veh/15-min)', color='black',size=15)
#     ax1.set_xticks(np.arange(0, 97, 4))
#     ax1.set_xticklabels(tt, rotation=90, fontsize=25)
#     ax1.set_xlim(0,96)
    #plt.gcf().autofmt_xdate()
    #ax1.xaxis.set_major_formatter(myFmt)
    plt.legend([bpr["boxes"][0],bpl["boxes"][0]], ['ANN', 'FSTGCN'], edgecolor='black', loc=9, prop={'size': 15})
    plt.legend(prop={'size': 15})
    plt.title("TMC (%s)" %(i),size=20)
    fig.savefig('C:\\Users\\szahedi1\\Photos\\belt\\(%s)TMC (%s).png'%(date,i))

# XGBoost - Intersection

In [ ]:
df.head()

In [ ]:
#df.to_csv('C:\\Users\\szahedi1\\Amir_Thesis\\Data.csv', sep=',')

In [ ]:
df = df.dropna()
df.shape

In [ ]:
df.groupby(['Detector_ID']).count()

In [ ]:
dum_df = pd.get_dummies(df, columns=["hour",'quarter','year','month','weekday','day','roadname_up','roadname_dn',
                                     'Detector_ID','dir','move','lanes_up','lanes_dn'] )
dh = df[['Detector_ID']]
dum_df = dum_df.join(dh).drop(['pair','measurement_tstamp','Up','Dn','Direction','xd_up','tmc_up','xd_dn','tmc_dn',
                               'Intersection Name','date','dn_dir','movements'],axis=1)
dum_df

In [ ]:
i=0
for loc in dum_df.Detector_ID.unique().tolist():
    i=i+1
    train=dum_df.loc[dum_df['Detector_ID']!=loc]
    test=dum_df.loc[dum_df['Detector_ID']==loc]
    
    X_train=train.drop(['count','Detector_ID'],axis=1)
    X_test=test.drop(['count','Detector_ID'],axis=1)
    
    y_train=train['count']
    y_test=test['count']
    
    regressor = xgb.XGBRegressor(n_estimators=100,reg_lambda=1, gamma=0,max_depth=3)
    regressor.fit(X_train, y_train)
    res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)
    if (i==1):
        results=res
    else:
        results=pd.concat([results,res])
if (len(results.loc[results['actual']-results['count']!=0])==0):
    print('Done')
else:
    print('there is a problem')

In [ ]:
# count
result = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAPE"])
for loc in dum_df.Detector_ID.unique().tolist():
    result=result.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'count'], results.loc[results['Detector_ID']==loc,'pred'] ),
                        "XGBoost_MAPE": (abs(results.loc[results['Detector_ID']==loc,'count']-results.loc[results['Detector_ID']==loc,'pred'])/results.loc[results['Detector_ID']==loc,'count']).mean()*100
                        },
                       ignore_index=True)
    
results['pro_act'] = results['count']/results['approach_count']
results['pro_est'] = results['pred']/results['approach_count']

# proportion
result_p = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAE"])
for loc in dum_df.Detector_ID.unique().tolist():
    result_p=result_p.append({"Detector_ID":str(loc),
        "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'pro_act'], results.loc[results['Detector_ID']==loc,'pro_est'] ),
                        "XGBoost_MAE": (abs(results.loc[results['Detector_ID']==loc,'pro_act']-results.loc[results['Detector_ID']==loc,'pro_est'])).mean()*100
                        },
                       ignore_index=True)

In [ ]:
results

In [ ]:
results['count_diff']    = results['pred']-results['count']
results['count_diff_h0'] = (results['approach_count']*results['pro_m'])-results['count']
results['abs_count_diff']    = abs(results['count_diff'])
results['abs_count_diff_h0'] = abs(results['count_diff_h0'])
results.abs_count_diff.mean()

In [ ]:
results.abs_count_diff_h0.mean()

In [ ]:
results['count'].mean()

In [ ]:
results[['Detector_ID','count_diff','abs_count_diff','abs_count_diff_h0','count']].groupby(['Detector_ID']).mean()

In [ ]:
['abs_count_diff']/results[['Detector_ID','count_diff','abs_count_diff','count']].groupby(['Detector_ID']).mean()['count']